In [33]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import preprocessing as scale
from sklearn.model_selection import train_test_split
from tqdm.notebook import tqdm
from sklearn import mixture
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

We do the repeat experiment

In [34]:
input_file = ["Backdoor.csv" , "DDoS_HTTP.csv" , "DDoS_ICMP.csv" , "DDoS_TCP.csv" , "DDoS_UDP.csv"  , "Password.csv" , "Port_Scanning.csv" ,"Ransomware.csv", "SQL_injection.csv" , "Uploading.csv" , "Vulnerability_scanner.csv" ,"XSS.csv" ]

The EMMC Function:

In [37]:
def pred(x, clfs, p_nodes, gms):
    p_x_node = np.zeros((len(x), len(clfs))) # p[x, in node i] = p(x | x in Node i) * p(x in Node i)
    p_y_given_node = np.zeros((len(x), len(clfs))) # p[y | x, node i]
    for i in range(len(clfs)):
        p_x_node[:, i] = p_nodes[i] * np.exp(gms[i].score_samples(x))
        p_y_given_node[:, i] = clfs[i].predict_proba(x)[:, 1]
    
    p_y = p_y_given_node * p_x_node / (np.sum(p_x_node, axis=-1, keepdims=True) + 1e-10)

    p_y = np.hstack([1-np.sum(p_y, axis=-1, keepdims=True), p_y])

    return np.argmax(p_y, axis=-1)

In [36]:
columns_to_drop_bi = ['Attack_type']

data_list = []
for i in range(len(input_file)):
    data = pd.read_csv(input_file[i])
    data['attack_type'] = i + 1
    data.loc[data.Attack_type == 'Normal', 'attack_type'] = 0
    data.drop(columns = columns_to_drop_bi, inplace = True)
    if len(data) < 2000:
        data_list.append(data)
    else:
        data0 = data[data['attack_type'].isin([0])]
        x1 = data0.sample(n = 1000, random_state = i + 42, axis = 0)
        data1 = data[data['attack_type'].isin([i+1])]
        x2 = data1.sample(n = 1000, random_state = i + 43, axis = 0)   
        data_selected = pd.concat([x1, x2], ignore_index=True)  
        data_list.append(data_selected) 
for i in range(len(data_list)):
    print(len(data_list[i]))  




2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000


In [17]:
data_list[9].tail()
# A sample of what we have

,arp.opcode,arp.hw.size,icmp.checksum,icmp.seq_le,http.content_length,http.response,tcp.ack,tcp.ack_raw,tcp.checksum,tcp.connection.fin,...,mqtt.conflag.cleansess,mqtt.conflags,mqtt.hdrflags,mqtt.len,mqtt.msgtype,mqtt.proto_len,mqtt.topic_len,mqtt.ver,Attack_label,attack_type
1995,0,0,0,0,0,0,150,1934832499,19721,0,...,0,0,0,0,0,0,0,0,1,10
1996,0,0,0,0,0,0,0,0,24507,0,...,0,0,0,0,0,0,0,0,1,10
1997,0,0,0,0,0,0,1,3703765364,46928,0,...,0,0,0,0,0,0,0,0,1,10
1998,0,0,0,0,0,0,1,208620284,44688,0,...,0,0,0,0,0,0,0,0,1,10
1999,0,0,0,0,0,0,1,3557485597,42863,0,...,0,0,0,0,0,0,0,0,1,10


In [5]:
rs = 42 # set the random state

In [6]:
from sklearn.svm import SVC
clfs = [] # This is going to contain 14 different classifiers
n_samples = []
x_train_list = []
y_train_list = []
x_test_list = []
y_test_list = []
y = [np.ones(len(data_list[i])) * (i + 1) for i in range(len(data_list))]
for i in tqdm(range(len(input_file))): # reading the data
    data0 = data_list[i]
    y[i][data0.attack_type == 0] = 0
    n_samples.append(len(y[i])) # the number of this node
    x = data_list[i]
    x = x.drop(columns='attack_type')
    x = x.drop(columns='Attack_label')
    x_train, x_test, y_train, y_test = train_test_split(x, y[i], test_size = 0.25, random_state = rs)
    x_train_list.append(x_train)
    y_train_list.append(y_train)
    x_test_list.append(x_test)
    y_test_list.append(y_test)

x_train_full = pd.concat(x_train_list, ignore_index=True)
scaler = scale.StandardScaler().fit(x_train_full)

for i in range(len(input_file)):
    x_train_list[i] = scaler.transform(x_train_list[i])

for i in tqdm(range(len(input_file))):
    #classifier = SVC(kernel = 'rbf', random_state = 41, gamma='scale',max_iter=-1, probability=True)
    # classifier = SVC(kernel = 'rbf', random_state = rs, gamma='scale', max_iter=-1, probability=True)
    # classifier.fit(data_list[i], y[i])
    classifier = LogisticRegression(max_iter=10000)
    classifier.fit(x_train_list[i], y_train_list[i])
    clfs.append(classifier)

total_n_samples = np.sum(n_samples)
print('total number is ',total_n_samples)
p_nodes =  np.array(n_samples) / total_n_samples
print('The proportion of the nodes are:', p_nodes)



total number is  24000
The proportion of the nodes are: [0.08333333 0.08333333 0.08333333 0.08333333 0.08333333 0.08333333
 0.08333333 0.08333333 0.08333333 0.08333333 0.08333333 0.08333333]


In [7]:
for i in range(len(input_file)):
    x_test_list[i] = scaler.transform(x_test_list[i])

In [8]:
x_test = np.vstack(x_test_list)
y_test = np.hstack(y_test_list)
x_train = np.vstack(x_train_list)
y_train = np.hstack(y_train_list)

In [24]:
# GMM
from sklearn.mixture import GaussianMixture
gms = []
K = 35 # The number of the components of the GMM, one could change it by analyzing the graph
for i in tqdm(range(len(input_file))):
    x = x_train_list[i]
    gm = GaussianMixture(n_components = K).fit(x)  
    gms.append(gm)

In [25]:

prediction = pred(x_test, clfs, p_nodes, gms)

In [26]:

correct = prediction == y_test
accuracy = np.mean(correct)
print('Overall accuracy is:', accuracy)

Overall accuracy is: 0.7766666666666666


In [27]:
accs = []
for i in range(len(input_file)+1):
    if len(correct[y_test==i]) == 0:
        accs.append(0)
    else:
        accs.append(np.mean(correct[y_test==i]))
print('Accuracies:', accs)
recalls = []
for i in range(len(input_file)+1):
    if len(correct[prediction==i]) == 0:
        recalls.append(0)
    else:
        recalls.append(np.mean(correct[prediction==i]))
print('Recalls:', recalls)

Accuracies: [0.9880025940337225, 0.7695473251028807, 0.30864197530864196, 0.9958847736625515, 0.7860082304526749, 1.0, 0.17695473251028807, 0.7366255144032922, 0.5925925925925926, 0.13580246913580246, 0.18930041152263374, 0.7860082304526749, 0.16049382716049382]
Recalls: [0.7410019455252919, 0.9166666666666666, 0.7211538461538461, 1.0, 0.7047970479704797, 1.0, 0.8431372549019608, 0.662962962962963, 0.9230769230769231, 0.6111111111111112, 0.7931034482758621, 0.9744897959183674, 1.0]


Repeat the test:

EMMC: Binary = LR


In [38]:
num_epoch = 20

recall , precision = np.ones((num_epoch,len(input_file)+1))*0 , np.ones((num_epoch,len(input_file)+1))*0

for epoch in tqdm(range(num_epoch)):
    #Data Process
    columns_to_drop_bi = ['Attack_type']

    data_list = []
    for i in range(len(input_file)):
        data = pd.read_csv(input_file[i])
        data['attack_type'] = i + 1 
        data.loc[data.Attack_type == 'Normal', 'attack_type'] = 0
        data.drop(columns = columns_to_drop_bi, inplace = True)

        # Random Pick:
        if len(data) < 2000:
            data_list.append(data)
        else:
            data0 = data[data['attack_type'].isin([0])]
            x1 = data0.sample(n = 1000, random_state = i + epoch, axis = 0)
            data1 = data[data['attack_type'].isin([i+1])]
            x2 = data1.sample(n = 1000, random_state = i + epoch + 1, axis = 0)   
            data_selected = pd.concat([x1, x2], ignore_index=True)  
            data_list.append(data_selected)  

    #Training

    clfs = [] # This is going to contain 12 different classifiers
    n_samples = []
    x_train_list = []
    y_train_list = []
    x_test_list = []
    y_test_list = []
    y = [np.ones(len(data_list[i])) * (i + 1) for i in range(len(data_list))]
    for i in range(len(input_file)): # reading the data
        data0 = data_list[i]
        y[i][data0.attack_type == 0] = 0
        n_samples.append(len(y[i])) # the number of this node
        x = data_list[i]
        x = x.drop(columns=['attack_type' , 'Attack_label'])
        # until now, we totally dropped the attack labels
        x_train, x_test, y_train, y_test = train_test_split(x, y[i], test_size = 0.25, random_state = epoch)
        x_train_list.append(x_train)
        y_train_list.append(y_train)
        x_test_list.append(x_test)
        y_test_list.append(y_test)

    x_train_full = pd.concat(x_train_list, ignore_index=True)
    scaler = scale.StandardScaler().fit(x_train_full)

    for i in range(len(input_file)):
        x_train_list[i] = scaler.transform(x_train_list[i])

    for i in range(len(input_file)):
        classifier = LogisticRegression(random_state=epoch)
        #classifier = KNeighborsClassifier()
        classifier.fit(x_train_list[i], y_train_list[i])
        clfs.append(classifier)

    total_n_samples = np.sum(n_samples)
    p_nodes =  np.array(n_samples) / total_n_samples

    # Finally, we train our GMM here
    from sklearn.mixture import GaussianMixture
    gms = []
    K = 30 # The number of the components of the GMM, one could change it by analyzing the graph
    for i in range(len(input_file)):
        x = x_train_list[i]
        gm = GaussianMixture(n_components = K).fit(x)  
        gms.append(gm)


    # Test:
    for i in range(len(input_file)):
        x_test_list[i] = scaler.transform(x_test_list[i])

    x_test = np.vstack(x_test_list)
    y_test = np.hstack(y_test_list)
    x_train = np.vstack(x_train_list)
    y_train = np.hstack(y_train_list)

    prediction = pred(x_test, clfs, p_nodes, gms)
    correct = prediction == y_test

    recs = []
    for i in range(len(input_file)+1):
        if len(correct[y_test==i]) == 0:
            recs.append(0)
        else:
            recs.append(np.mean(correct[y_test==i]))
    
    recall[epoch,:] = recs

    precs = []
    for i in range(len(input_file)+1):
        if len(correct[prediction==i]) == 0:
            precs.append(0)
        else:
            precs.append(np.mean(correct[prediction==i]))
    
    precision[epoch,:] = precs
# For Latex:

print('Latex Precisions')
for i in range(len(input_file)+1):
      print(round(100 * np.mean(precision[:,i]),2),'\% (',round(np.std(precision[:,i]),2),') &' )


print('Latex Recalls')
for i in range(len(input_file)+1):
      print(round(100 * np.mean(recall[:,i]),2),'\% (',round(np.std(recall[:,i]),2),') &' )


Latex Precisions
71.84 \% ( 0.01 ) &
92.16 \% ( 0.05 ) &
83.02 \% ( 0.13 ) &
100.0 \% ( 0.0 ) &
73.04 \% ( 0.03 ) &
100.0 \% ( 0.0 ) &
82.01 \% ( 0.06 ) &
67.4 \% ( 0.04 ) &
88.4 \% ( 0.06 ) &
80.41 \% ( 0.13 ) &
72.72 \% ( 0.12 ) &
96.79 \% ( 0.02 ) &
92.26 \% ( 0.09 ) &
Latex precs
99.1 \% ( 0.0 ) &
71.3 \% ( 0.04 ) &
33.81 \% ( 0.03 ) &
100.0 \% ( 0.0 ) &
78.02 \% ( 0.03 ) &
99.96 \% ( 0.0 ) &
15.12 \% ( 0.03 ) &
72.68 \% ( 0.04 ) &
46.61 \% ( 0.09 ) &
14.31 \% ( 0.03 ) &
17.59 \% ( 0.03 ) &
78.87 \% ( 0.03 ) &
14.37 \% ( 0.04 ) &


binary = KNN

In [39]:
num_epoch = 20

recall , precision = np.ones((num_epoch,len(input_file)+1))*0 , np.ones((num_epoch,len(input_file)+1))*0

for epoch in tqdm(range(num_epoch)):
    #Data Process
    columns_to_drop_bi = ['Attack_type']

    data_list = []
    for i in range(len(input_file)):
        data = pd.read_csv(input_file[i])
        data['attack_type'] = i + 1 
        data.loc[data.Attack_type == 'Normal', 'attack_type'] = 0
        data.drop(columns = columns_to_drop_bi, inplace = True)

        # Random Pick:
        if len(data) < 2000:
            data_list.append(data)
        else:
            data0 = data[data['attack_type'].isin([0])]
            x1 = data0.sample(n = 1000, random_state = i + epoch, axis = 0)
            data1 = data[data['attack_type'].isin([i+1])]
            x2 = data1.sample(n = 1000, random_state = i + epoch + 1, axis = 0)   
            data_selected = pd.concat([x1, x2], ignore_index=True)  
            data_list.append(data_selected)  

    #Training

    clfs = [] # This is going to contain 12 different classifiers
    n_samples = []
    x_train_list = []
    y_train_list = []
    x_test_list = []
    y_test_list = []
    y = [np.ones(len(data_list[i])) * (i + 1) for i in range(len(data_list))]
    for i in range(len(input_file)): # reading the data
        data0 = data_list[i]
        y[i][data0.attack_type == 0] = 0
        n_samples.append(len(y[i])) # the number of this node
        x = data_list[i]
        x = x.drop(columns=['attack_type' , 'Attack_label'])
        # until now, we totally dropped the attack labels
        x_train, x_test, y_train, y_test = train_test_split(x, y[i], test_size = 0.25, random_state = epoch)
        x_train_list.append(x_train)
        y_train_list.append(y_train)
        x_test_list.append(x_test)
        y_test_list.append(y_test)

    x_train_full = pd.concat(x_train_list, ignore_index=True)
    scaler = scale.StandardScaler().fit(x_train_full)

    for i in range(len(input_file)):
        x_train_list[i] = scaler.transform(x_train_list[i])

    for i in range(len(input_file)):
        classifier = KNeighborsClassifier()
        classifier.fit(x_train_list[i], y_train_list[i])
        clfs.append(classifier)

    total_n_samples = np.sum(n_samples)
    p_nodes =  np.array(n_samples) / total_n_samples

    # Finally, we train our GMM here
    from sklearn.mixture import GaussianMixture
    gms = []
    K = 30 # The number of the components of the GMM, one could change it by analyzing the graph
    for i in range(len(input_file)):
        x = x_train_list[i]
        gm = GaussianMixture(n_components = K).fit(x)  
        gms.append(gm)


    # Test:
    for i in range(len(input_file)):
        x_test_list[i] = scaler.transform(x_test_list[i])

    x_test = np.vstack(x_test_list)
    y_test = np.hstack(y_test_list)
    x_train = np.vstack(x_train_list)
    y_train = np.hstack(y_train_list)

    prediction = pred(x_test, clfs, p_nodes, gms)
    correct = prediction == y_test

    recs = []
    for i in range(len(input_file)+1):
        if len(correct[y_test==i]) == 0:
            recs.append(0)
        else:
            recs.append(np.mean(correct[y_test==i]))
    
    recall[epoch,:] = recs

    precs = []
    for i in range(len(input_file)+1):
        if len(correct[prediction==i]) == 0:
            precs.append(0)
        else:
            precs.append(np.mean(correct[prediction==i]))
    
    precision[epoch,:] = precs
# For Latex:

print('Latex Precisions')
for i in range(len(input_file)+1):
      print(round(100 * np.mean(precision[:,i]),2),'\% (',round(np.std(precision[:,i]),2),') &' )


print('Latex Recalls')
for i in range(len(input_file)+1):
      print(round(100 * np.mean(recall[:,i]),2),'\% (',round(np.std(recall[:,i]),2),') &' )


Latex Precisions
73.18 \% ( 0.01 ) &
91.36 \% ( 0.06 ) &
78.79 \% ( 0.12 ) &
100.0 \% ( 0.0 ) &
77.64 \% ( 0.03 ) &
100.0 \% ( 0.0 ) &
78.08 \% ( 0.1 ) &
65.66 \% ( 0.03 ) &
76.79 \% ( 0.06 ) &
77.46 \% ( 0.1 ) &
70.68 \% ( 0.1 ) &
98.3 \% ( 0.01 ) &
82.33 \% ( 0.14 ) &
Latex Recalls
98.83 \% ( 0.0 ) &
76.33 \% ( 0.03 ) &
34.76 \% ( 0.03 ) &
99.96 \% ( 0.0 ) &
74.52 \% ( 0.04 ) &
99.74 \% ( 0.0 ) &
14.3 \% ( 0.02 ) &
77.9 \% ( 0.04 ) &
57.65 \% ( 0.09 ) &
15.41 \% ( 0.03 ) &
16.62 \% ( 0.03 ) &
78.36 \% ( 0.02 ) &
15.42 \% ( 0.02 ) &


SVM Binary

In [40]:
num_epoch = 20

recall , precision = np.ones((num_epoch,len(input_file)+1))*0 , np.ones((num_epoch,len(input_file)+1))*0

for epoch in tqdm(range(num_epoch)):
    #Data Process
    columns_to_drop_bi = ['Attack_type']

    data_list = []
    for i in range(len(input_file)):
        data = pd.read_csv(input_file[i])
        data['attack_type'] = i + 1 
        data.loc[data.Attack_type == 'Normal', 'attack_type'] = 0
        data.drop(columns = columns_to_drop_bi, inplace = True)

        # Random Pick:
        if len(data) < 2000:
            data_list.append(data)
        else:
            data0 = data[data['attack_type'].isin([0])]
            x1 = data0.sample(n = 1000, random_state = i + epoch, axis = 0)
            data1 = data[data['attack_type'].isin([i+1])]
            x2 = data1.sample(n = 1000, random_state = i + epoch + 1, axis = 0)   
            data_selected = pd.concat([x1, x2], ignore_index=True)  
            data_list.append(data_selected)  

    #Training

    clfs = [] # This is going to contain 12 different classifiers
    n_samples = []
    x_train_list = []
    y_train_list = []
    x_test_list = []
    y_test_list = []
    y = [np.ones(len(data_list[i])) * (i + 1) for i in range(len(data_list))]
    for i in range(len(input_file)): # reading the data
        data0 = data_list[i]
        y[i][data0.attack_type == 0] = 0
        n_samples.append(len(y[i])) # the number of this node
        x = data_list[i]
        x = x.drop(columns=['attack_type' , 'Attack_label'])
        # until now, we totally dropped the attack labels
        x_train, x_test, y_train, y_test = train_test_split(x, y[i], test_size = 0.25, random_state = epoch)
        x_train_list.append(x_train)
        y_train_list.append(y_train)
        x_test_list.append(x_test)
        y_test_list.append(y_test)

    x_train_full = pd.concat(x_train_list, ignore_index=True)
    scaler = scale.StandardScaler().fit(x_train_full)

    for i in range(len(input_file)):
        x_train_list[i] = scaler.transform(x_train_list[i])

    for i in range(len(input_file)):
        classifier = SVC(kernel = 'rbf', random_state = 41 + epoch, gamma='scale',max_iter=-1, probability=True)
        classifier.fit(x_train_list[i], y_train_list[i])
        clfs.append(classifier)

    total_n_samples = np.sum(n_samples)
    p_nodes =  np.array(n_samples) / total_n_samples

    # Finally, we train our GMM here
    from sklearn.mixture import GaussianMixture
    gms = []
    K = 30 # The number of the components of the GMM, one could change it by analyzing the graph
    for i in range(len(input_file)):
        x = x_train_list[i]
        gm = GaussianMixture(n_components = K).fit(x)  
        gms.append(gm)


    # Test:
    for i in range(len(input_file)):
        x_test_list[i] = scaler.transform(x_test_list[i])

    x_test = np.vstack(x_test_list)
    y_test = np.hstack(y_test_list)
    x_train = np.vstack(x_train_list)
    y_train = np.hstack(y_train_list)

    prediction = pred(x_test, clfs, p_nodes, gms)
    correct = prediction == y_test

    recs = []
    for i in range(len(input_file)+1):
        if len(correct[y_test==i]) == 0:
            recs.append(0)
        else:
            recs.append(np.mean(correct[y_test==i]))
    
    recall[epoch,:] = recs

    precs = []
    for i in range(len(input_file)+1):
        if len(correct[prediction==i]) == 0:
            precs.append(0)
        else:
            precs.append(np.mean(correct[prediction==i]))
    
    precision[epoch,:] = precs
# For Latex:

print('Latex Precisions')
for i in range(len(input_file)+1):
      print(round(100 * np.mean(precision[:,i]),2),'\% (',round(np.std(precision[:,i]),2),') &' )


print('Latex Recalls')
for i in range(len(input_file)+1):
      print(round(100 * np.mean(recall[:,i]),2),'\% (',round(np.std(recall[:,i]),2),') &' )


Latex Precisions
72.75 \% ( 0.01 ) &
90.44 \% ( 0.07 ) &
74.55 \% ( 0.15 ) &
100.0 \% ( 0.0 ) &
77.14 \% ( 0.05 ) &
100.0 \% ( 0.0 ) &
83.26 \% ( 0.06 ) &
65.52 \% ( 0.04 ) &
75.16 \% ( 0.08 ) &
82.44 \% ( 0.11 ) &
70.2 \% ( 0.1 ) &
96.94 \% ( 0.03 ) &
84.75 \% ( 0.17 ) &
Latex Recalls
98.63 \% ( 0.01 ) &
74.73 \% ( 0.05 ) &
33.83 \% ( 0.03 ) &
100.0 \% ( 0.0 ) &
74.9 \% ( 0.05 ) &
99.96 \% ( 0.0 ) &
13.84 \% ( 0.03 ) &
78.44 \% ( 0.06 ) &
51.73 \% ( 0.1 ) &
14.54 \% ( 0.03 ) &
16.57 \% ( 0.03 ) &
78.94 \% ( 0.02 ) &
14.19 \% ( 0.04 ) &


RF binary

In [41]:
num_epoch = 20

recall , precision = np.ones((num_epoch,len(input_file)+1))*0 , np.ones((num_epoch,len(input_file)+1))*0

for epoch in tqdm(range(num_epoch)):
    #Data Process
    columns_to_drop_bi = ['Attack_type']

    data_list = []
    for i in range(len(input_file)):
        data = pd.read_csv(input_file[i])
        data['attack_type'] = i + 1 
        data.loc[data.Attack_type == 'Normal', 'attack_type'] = 0
        data.drop(columns = columns_to_drop_bi, inplace = True)

        # Random Pick:
        if len(data) < 2000:
            data_list.append(data)
        else:
            data0 = data[data['attack_type'].isin([0])]
            x1 = data0.sample(n = 1000, random_state = i + epoch, axis = 0)
            data1 = data[data['attack_type'].isin([i+1])]
            x2 = data1.sample(n = 1000, random_state = i + epoch + 1, axis = 0)   
            data_selected = pd.concat([x1, x2], ignore_index=True)  
            data_list.append(data_selected)  

    #Training

    clfs = [] # This is going to contain 12 different classifiers
    n_samples = []
    x_train_list = []
    y_train_list = []
    x_test_list = []
    y_test_list = []
    y = [np.ones(len(data_list[i])) * (i + 1) for i in range(len(data_list))]
    for i in range(len(input_file)): # reading the data
        data0 = data_list[i]
        y[i][data0.attack_type == 0] = 0
        n_samples.append(len(y[i])) # the number of this node
        x = data_list[i]
        x = x.drop(columns=['attack_type' , 'Attack_label'])
        # until now, we totally dropped the attack labels
        x_train, x_test, y_train, y_test = train_test_split(x, y[i], test_size = 0.25, random_state = epoch)
        x_train_list.append(x_train)
        y_train_list.append(y_train)
        x_test_list.append(x_test)
        y_test_list.append(y_test)

    x_train_full = pd.concat(x_train_list, ignore_index=True)
    scaler = scale.StandardScaler().fit(x_train_full)

    for i in range(len(input_file)):
        x_train_list[i] = scaler.transform(x_train_list[i])

    for i in range(len(input_file)):
        classifier = RandomForestClassifier(n_estimators = 10, criterion = 'entropy', random_state = rs)
        classifier.fit(x_train_list[i], y_train_list[i])
        clfs.append(classifier)

    total_n_samples = np.sum(n_samples)
    p_nodes =  np.array(n_samples) / total_n_samples

    # Finally, we train our GMM here
    from sklearn.mixture import GaussianMixture
    gms = []
    K = 30 # The number of the components of the GMM, one could change it by analyzing the graph
    for i in range(len(input_file)):
        x = x_train_list[i]
        gm = GaussianMixture(n_components = K).fit(x)  
        gms.append(gm)


    # Test:
    for i in range(len(input_file)):
        x_test_list[i] = scaler.transform(x_test_list[i])

    x_test = np.vstack(x_test_list)
    y_test = np.hstack(y_test_list)
    x_train = np.vstack(x_train_list)
    y_train = np.hstack(y_train_list)

    prediction = pred(x_test, clfs, p_nodes, gms)
    correct = prediction == y_test

    recs = []
    for i in range(len(input_file)+1):
        if len(correct[y_test==i]) == 0:
            recs.append(0)
        else:
            recs.append(np.mean(correct[y_test==i]))
    
    recall[epoch,:] = recs

    precs = []
    for i in range(len(input_file)+1):
        if len(correct[prediction==i]) == 0:
            precs.append(0)
        else:
            precs.append(np.mean(correct[prediction==i]))
    
    precision[epoch,:] = precs
# For Latex:

print('Latex Precisions')
for i in range(len(input_file)+1):
      print(round(100 * np.mean(precision[:,i]),2),'\% (',round(np.std(precision[:,i]),2),') &' )


print('Latex Recalls')
for i in range(len(input_file)+1):
      print(round(100 * np.mean(recall[:,i]),2),'\% (',round(np.std(recall[:,i]),2),') &' )


Latex Precisions
74.98 \% ( 0.02 ) &
94.51 \% ( 0.05 ) &
71.11 \% ( 0.1 ) &
100.0 \% ( 0.0 ) &
98.2 \% ( 0.03 ) &
100.0 \% ( 0.0 ) &
71.92 \% ( 0.16 ) &
69.45 \% ( 0.04 ) &
79.61 \% ( 0.05 ) &
71.16 \% ( 0.12 ) &
68.7 \% ( 0.1 ) &
98.3 \% ( 0.02 ) &
67.95 \% ( 0.16 ) &
Latex Recalls
99.11 \% ( 0.0 ) &
78.14 \% ( 0.04 ) &
35.7 \% ( 0.04 ) &
99.96 \% ( 0.0 ) &
78.42 \% ( 0.07 ) &
99.9 \% ( 0.0 ) &
16.39 \% ( 0.03 ) &
92.46 \% ( 0.03 ) &
66.04 \% ( 0.12 ) &
20.85 \% ( 0.05 ) &
17.14 \% ( 0.03 ) &
78.48 \% ( 0.02 ) &
15.49 \% ( 0.03 ) &


Full data classification:

LR

In [42]:
num_epoch = 20

recall , precision = np.ones((num_epoch,len(input_file)+1))*0 , np.ones((num_epoch,len(input_file)+1))*0

for epoch in tqdm(range(num_epoch)):
    #Data Process
    columns_to_drop_bi = ['Attack_type']

    data_list = []
    for i in range(len(input_file)):
        data = pd.read_csv(input_file[i])
        data['attack_type'] = i + 1
        data.loc[data.Attack_type == 'Normal', 'attack_type'] = 0
        data.drop(columns = columns_to_drop_bi, inplace = True)
        if len(data) < 2000:
            data_list.append(data)
        else:
            data0 = data[data['attack_type'].isin([0])]
            x1 = data0.sample(n = 1000, random_state = i + epoch, axis = 0)
            data1 = data[data['attack_type'].isin([i+1])]
            x2 = data1.sample(n = 1000, random_state = i + epoch + 1, axis = 0)   
            data_selected = pd.concat([x1, x2], ignore_index=True)  
            data_list.append(data_selected)  
    #Training


    clfs = [] # This is going to contain 12 different classifiers
    n_samples = []
    x_train_list = []
    y_train_list = []
    x_test_list = []
    y_test_list = []
    y = [np.ones(len(data_list[i])) * (i + 1) for i in range(len(data_list))]
    for i in range(len(input_file)): # reading the data
        data0 = data_list[i]
        y[i][data0.attack_type == 0] = 0
        n_samples.append(len(y[i])) # the number of this node
        x = data_list[i]
        x = x.drop(columns=['attack_type','Attack_label'])
        x_train, x_test, y_train, y_test = train_test_split(x, y[i], test_size = 0.25, random_state = epoch+40)
        x_train_list.append(x_train)
        y_train_list.append(y_train)
        x_test_list.append(x_test)
        y_test_list.append(y_test)

    x_train_full = pd.concat(x_train_list, ignore_index=True)
    y_train_full = np.hstack(y_train_list)
    x_test_full = pd.concat(x_test_list, ignore_index=True)
    #y_test_full = pd.concat(y_test_list, ignore_index=True)
    scaler = scale.StandardScaler().fit(x_train_full)
    x_train_full = scaler.transform(x_train_full)
    classifier = LogisticRegression(random_state = epoch)
    classifier.fit(x_train_full, y_train_full)

    x_test_full = scaler.transform(x_test_full)

    #x_test = np.vstack(x_test_list)
    y_test_full = np.hstack(y_test_list)
   # x_train = np.vstack(x_train_list)
    


    prediction = classifier.predict(x_test_full)
    correct = prediction == y_test_full

    rec = []
    for i in range(len(input_file)+1):
        if len(correct[y_test_full==i]) == 0:
            rec.append(0)
        else:
            rec.append(np.mean(correct[y_test_full==i]))
    
    recall[epoch,:] = rec
      

    pr = []
    for i in range(len(input_file)+1):
        if len(correct[prediction==i]) == 0:
            pr.append(0)
        else:
            pr.append(np.mean(correct[prediction==i]))
    
    precision[epoch,:] = pr
    
for i in range(len(input_file)+1):
      print(i,'Precision mean=',np.mean(precision[:,i]))
      print(i,'Precision std=',np.std(precision[:,i]))

for i in range(len(input_file)+1):
      print('Recall mean of ',i,' =',np.mean(recall[:,i]))
      print('Recall std of ',i,' =',np.std(recall[:,i]))
      
print('Latex Precisions')
for i in range(len(input_file)+1):
      print(round(100 * np.mean(precision[:,i]),2),'\% (',round(np.std(precision[:,i]),2),') &' )


print('Latex Recalls')
for i in range(len(input_file)+1):
      print(round(100 * np.mean(recall[:,i]),2),'\% (',round(np.std(recall[:,i]),2),') &' )


/Users/irohayachiyo/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/irohayachiyo/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_mo


0 Precision mean= 0.6972837585359323
0 Precision std= 0.018303512465238474
1 Precision mean= 0.4038418216092398
1 Precision std= 0.03937783178100957
2 Precision mean= 0.527915416028818
2 Precision std= 0.04127416591604016
3 Precision mean= 0.9986146263616664
3 Precision std= 0.002886313340697261
4 Precision mean= 0.7418605439876964
4 Precision std= 0.06688044980148687
5 Precision mean= 0.9993941852425012
5 Precision std= 0.001442356923587901
6 Precision mean= 0.8206673433801329
6 Precision std= 0.14557647066506124
7 Precision mean= 0.6777563878212962
7 Precision std= 0.06208623882638545
8 Precision mean= 0.3309501884819273
8 Precision std= 0.05465581264530748
9 Precision mean= 0.49680769876716424
9 Precision std= 0.0770678917169437
10 Precision mean= 0.6444430145872899
10 Precision std= 0.1020292679184349
11 Precision mean= 0.8785379113370929
11 Precision std= 0.01608047448712708
12 Precision mean= 0.0
12 Precision std= 0.0
Recall mean of  0  = 0.9908181767574199
Recall std of  0  = 0

/Users/irohayachiyo/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


KNN Full

In [41]:
num_epoch = 20

recall , precision = np.ones((num_epoch,len(input_file)+1))*0 , np.ones((num_epoch,len(input_file)+1))*0

for epoch in tqdm(range(num_epoch)):
    #Data Process
    columns_to_drop_bi = ['Attack_type']

    data_list = []
    for i in range(len(input_file)):
        data = pd.read_csv(input_file[i])
        data['attack_type'] = i + 1
        data.loc[data.Attack_type == 'Normal', 'attack_type'] = 0
        data.drop(columns = columns_to_drop_bi, inplace = True)
        if len(data) < 2000:
            data_list.append(data)
        else:
            data0 = data[data['attack_type'].isin([0])]
            x1 = data0.sample(n = 1000, random_state = i + epoch, axis = 0)
            data1 = data[data['attack_type'].isin([i+1])]
            x2 = data1.sample(n = 1000, random_state = i + epoch + 1, axis = 0)   
            data_selected = pd.concat([x1, x2], ignore_index=True)  
            data_list.append(data_selected)  
    #Training


    clfs = [] # This is going to contain 12 different classifiers
    n_samples = []
    x_train_list = []
    y_train_list = []
    x_test_list = []
    y_test_list = []
    y = [np.ones(len(data_list[i])) * (i + 1) for i in range(len(data_list))]
    for i in range(len(input_file)): # reading the data
        data0 = data_list[i]
        y[i][data0.attack_type == 0] = 0
        n_samples.append(len(y[i])) # the number of this node
        x = data_list[i]
        x = x.drop(columns=['attack_type','Attack_label'])
        x_train, x_test, y_train, y_test = train_test_split(x, y[i], test_size = 0.25, random_state = epoch+40)
        x_train_list.append(x_train)
        y_train_list.append(y_train)
        x_test_list.append(x_test)
        y_test_list.append(y_test)

    x_train_full = pd.concat(x_train_list, ignore_index=True)
    y_train_full = np.hstack(y_train_list)
    x_test_full = pd.concat(x_test_list, ignore_index=True)
    #y_test_full = pd.concat(y_test_list, ignore_index=True)
    scaler = scale.StandardScaler().fit(x_train_full)
    x_train_full = scaler.transform(x_train_full)
    classifier = KNeighborsClassifier()
    classifier.fit(x_train_full, y_train_full)

    x_test_full = scaler.transform(x_test_full)

    #x_test = np.vstack(x_test_list)
    y_test_full = np.hstack(y_test_list)
   # x_train = np.vstack(x_train_list)
    


    prediction = classifier.predict(x_test_full)
    correct = prediction == y_test_full

    rec = []
    for i in range(len(input_file)+1):
        if len(correct[y_test_full==i]) == 0:
            rec.append(0)
        else:
            rec.append(np.mean(correct[y_test_full==i]))
    
    recall[epoch,:] = rec
      

    pr = []
    for i in range(len(input_file)+1):
        if len(correct[prediction==i]) == 0:
            pr.append(0)
        else:
            pr.append(np.mean(correct[prediction==i]))
    
    precision[epoch,:] = pr
    
for i in range(len(input_file)+1):
      print(i,'Precision mean=',np.mean(precision[:,i]))
      print(i,'Precision std=',np.std(precision[:,i]))

for i in range(len(input_file)+1):
      print('Recall mean of ',i,' =',np.mean(recall[:,i]))
      print('Recall std of ',i,' =',np.std(recall[:,i]))
      
print('Latex Precisions')
for i in range(len(input_file)+1):
      print(round(100 * np.mean(precision[:,i]),2),'\% (',round(np.std(precision[:,i]),2),') &' )


print('Latex Recalls')
for i in range(len(input_file)+1):
      print(round(100 * np.mean(recall[:,i]),2),'\% (',round(np.std(recall[:,i]),2),') &' )



0 Precision mean= 0.7480887457555241
0 Precision std= 0.016593364965375768
1 Precision mean= 0.5419835949755898
1 Precision std= 0.028222316490708946
2 Precision mean= 0.48929620907957927
2 Precision std= 0.03691055437913015
3 Precision mean= 0.9962746197645146
3 Precision std= 0.0035993217922279234
4 Precision mean= 0.7095183709391046
4 Precision std= 0.02224567467586668
5 Precision mean= 0.9980016885928794
5 Precision std= 0.0032779936852438754
6 Precision mean= 0.33120715558394476
6 Precision std= 0.07247355892328458
7 Precision mean= 0.7035021999864176
7 Precision std= 0.027260892697049437
8 Precision mean= 0.687061573274929
8 Precision std= 0.03701352708301233
9 Precision mean= 0.4018288689149343
9 Precision std= 0.05541492568783841
10 Precision mean= 0.38269939622721905
10 Precision std= 0.04870929614910959
11 Precision mean= 0.8470160659294919
11 Precision std= 0.022105486316875343
12 Precision mean= 0.20763897919367
12 Precision std= 0.05495894102530598
Recall mean of  0  = 0.

RF Full

In [48]:
num_epoch = 20

recall , precision = np.ones((num_epoch,len(input_file)+1))*0 , np.ones((num_epoch,len(input_file)+1))*0

for epoch in tqdm(range(num_epoch)):
    #Data Process
    columns_to_drop_bi = ['Attack_type']

    data_list = []
    for i in range(len(input_file)):
        data = pd.read_csv(input_file[i])
        data['attack_type'] = i + 1
        data.loc[data.Attack_type == 'Normal', 'attack_type'] = 0
        data.drop(columns = columns_to_drop_bi, inplace = True)
        if len(data) < 2000:
            data_list.append(data)
        else:
            data0 = data[data['attack_type'].isin([0])]
            x1 = data0.sample(n = 1000, random_state = i + epoch, axis = 0)
            data1 = data[data['attack_type'].isin([i+1])]
            x2 = data1.sample(n = 1000, random_state = i + epoch + 1, axis = 0)   
            data_selected = pd.concat([x1, x2], ignore_index=True)  
            data_list.append(data_selected)  
    #Training


    clfs = [] # This is going to contain 12 different classifiers
    n_samples = []
    x_train_list = []
    y_train_list = []
    x_test_list = []
    y_test_list = []
    y = [np.ones(len(data_list[i])) * (i + 1) for i in range(len(data_list))]
    for i in range(len(input_file)): # reading the data
        data0 = data_list[i]
        y[i][data0.attack_type == 0] = 0
        n_samples.append(len(y[i])) # the number of this node
        x = data_list[i]
        x = x.drop(columns=['attack_type','Attack_label'])
        x_train, x_test, y_train, y_test = train_test_split(x, y[i], test_size = 0.25, random_state = epoch+40)
        x_train_list.append(x_train)
        y_train_list.append(y_train)
        x_test_list.append(x_test)
        y_test_list.append(y_test)

    x_train_full = pd.concat(x_train_list, ignore_index=True)
    y_train_full = np.hstack(y_train_list)
    x_test_full = pd.concat(x_test_list, ignore_index=True)
    #y_test_full = pd.concat(y_test_list, ignore_index=True)
    scaler = scale.StandardScaler().fit(x_train_full)
    x_train_full = scaler.transform(x_train_full)
    classifier = RandomForestClassifier(n_estimators = 10, criterion = 'entropy', random_state = rs + i)
    classifier.fit(x_train_full, y_train_full)

    x_test_full = scaler.transform(x_test_full)

    #x_test = np.vstack(x_test_list)
    y_test_full = np.hstack(y_test_list)
   # x_train = np.vstack(x_train_list)
    


    prediction = classifier.predict(x_test_full)
    correct = prediction == y_test_full

    rec = []
    for i in range(len(input_file)+1):
        if len(correct[y_test_full==i]) == 0:
            rec.append(0)
        else:
            rec.append(np.mean(correct[y_test_full==i]))
    
    recall[epoch,:] = rec
      

    pr = []
    for i in range(len(input_file)+1):
        if len(correct[prediction==i]) == 0:
            pr.append(0)
        else:
            pr.append(np.mean(correct[prediction==i]))
    
    precision[epoch,:] = pr
    
for i in range(len(input_file)+1):
      print(i,'Precision mean=',np.mean(precision[:,i]))
      print(i,'Precision std=',np.std(precision[:,i]))

for i in range(len(input_file)+1):
      print('Recall mean of ',i,' =',np.mean(recall[:,i]))
      print('Recall std of ',i,' =',np.std(recall[:,i]))
      
print('Latex Precisions')
for i in range(len(input_file)+1):
      print(round(100 * np.mean(precision[:,i]),2),'\% (',round(np.std(precision[:,i]),2),') &' )


print('Latex Recalls')
for i in range(len(input_file)+1):
      print(round(100 * np.mean(recall[:,i]),2),'\% (',round(np.std(recall[:,i]),2),') &' )



0 Precision mean= 0.90589853888143
0 Precision std= 0.006580613078238353
1 Precision mean= 0.9366169078043483
1 Precision std= 0.014485752411366626
2 Precision mean= 0.6415434786173213
2 Precision std= 0.02361746883293334
3 Precision mean= 0.9995997694672131
3 Precision std= 0.0012010757582163777
4 Precision mean= 0.9997863247863247
4 Precision std= 0.0009313886631497133
5 Precision mean= 0.9985836138618666
5 Precision std= 0.0030048836765764414
6 Precision mean= 0.8113134230011194
6 Precision std= 0.02879803513308709
7 Precision mean= 0.8307061444642816
7 Precision std= 0.021755638790884855
8 Precision mean= 0.8064086597578095
8 Precision std= 0.03755852021785365
9 Precision mean= 0.7592343586622707
9 Precision std= 0.034898868632500524
10 Precision mean= 0.6729387806635958
10 Precision std= 0.025266515705357595
11 Precision mean= 0.9636122397297683
11 Precision std= 0.010653571517084418
12 Precision mean= 0.8107260470177945
12 Precision std= 0.04936101341480204
Recall mean of  0  = 

SVM Full

In [49]:
num_epoch = 20

recall , precision = np.ones((num_epoch,len(input_file)+1))*0 , np.ones((num_epoch,len(input_file)+1))*0

for epoch in tqdm(range(num_epoch)):
    #Data Process
    columns_to_drop_bi = ['Attack_type']

    data_list = []
    for i in range(len(input_file)):
        data = pd.read_csv(input_file[i])
        data['attack_type'] = i + 1
        data.loc[data.Attack_type == 'Normal', 'attack_type'] = 0
        data.drop(columns = columns_to_drop_bi, inplace = True)
        if len(data) < 2000:
            data_list.append(data)
        else:
            data0 = data[data['attack_type'].isin([0])]
            x1 = data0.sample(n = 1000, random_state = i + epoch, axis = 0)
            data1 = data[data['attack_type'].isin([i+1])]
            x2 = data1.sample(n = 1000, random_state = i + epoch + 1, axis = 0)   
            data_selected = pd.concat([x1, x2], ignore_index=True)  
            data_list.append(data_selected)  
    #Training


    clfs = [] # This is going to contain 12 different classifiers
    n_samples = []
    x_train_list = []
    y_train_list = []
    x_test_list = []
    y_test_list = []
    y = [np.ones(len(data_list[i])) * (i + 1) for i in range(len(data_list))]
    for i in range(len(input_file)): # reading the data
        data0 = data_list[i]
        y[i][data0.attack_type == 0] = 0
        n_samples.append(len(y[i])) # the number of this node
        x = data_list[i]
        x = x.drop(columns=['attack_type','Attack_label'])
        x_train, x_test, y_train, y_test = train_test_split(x, y[i], test_size = 0.25, random_state = epoch+40)
        x_train_list.append(x_train)
        y_train_list.append(y_train)
        x_test_list.append(x_test)
        y_test_list.append(y_test)

    x_train_full = pd.concat(x_train_list, ignore_index=True)
    y_train_full = np.hstack(y_train_list)
    x_test_full = pd.concat(x_test_list, ignore_index=True)
    #y_test_full = pd.concat(y_test_list, ignore_index=True)
    scaler = scale.StandardScaler().fit(x_train_full)
    x_train_full = scaler.transform(x_train_full)
    classifier = SVC(kernel = 'rbf', random_state = 41, gamma='scale',max_iter=-1, probability=True)
    classifier.fit(x_train_full, y_train_full)

    x_test_full = scaler.transform(x_test_full)

    #x_test = np.vstack(x_test_list)
    y_test_full = np.hstack(y_test_list)
   # x_train = np.vstack(x_train_list)
    


    prediction = classifier.predict(x_test_full)
    correct = prediction == y_test_full

    rec = []
    for i in range(len(input_file)+1):
        if len(correct[y_test_full==i]) == 0:
            rec.append(0)
        else:
            rec.append(np.mean(correct[y_test_full==i]))
    
    recall[epoch,:] = rec
      

    pr = []
    for i in range(len(input_file)+1):
        if len(correct[prediction==i]) == 0:
            pr.append(0)
        else:
            pr.append(np.mean(correct[prediction==i]))
    
    precision[epoch,:] = pr
    
for i in range(len(input_file)+1):
      print(i,'Precision mean=',np.mean(precision[:,i]))
      print(i,'Precision std=',np.std(precision[:,i]))

for i in range(len(input_file)+1):
      print('Recall mean of ',i,' =',np.mean(recall[:,i]))
      print('Recall std of ',i,' =',np.std(recall[:,i]))
      
print('Latex Precisions')
for i in range(len(input_file)+1):
      print(round(100 * np.mean(precision[:,i]),2),'\% (',round(np.std(precision[:,i]),2),') &' )


print('Latex Recalls')
for i in range(len(input_file)+1):
      print(round(100 * np.mean(recall[:,i]),2),'\% (',round(np.std(recall[:,i]),2),') &' )



0 Precision mean= 0.7018560977899171
0 Precision std= 0.01823249266520187
1 Precision mean= 0.4211066053685132
1 Precision std= 0.0765853174386387
2 Precision mean= 0.4711404864775954
2 Precision std= 0.04005645546023005
3 Precision mean= 0.9966277564913211
3 Precision std= 0.003125778608181801
4 Precision mean= 0.9524019413331141
4 Precision std= 0.10602799176390985
5 Precision mean= 0.8435448069392588
5 Precision std= 0.054839739133382644
6 Precision mean= 0.9898793363499246
6 Precision std= 0.024262373302972276
7 Precision mean= 0.5981090682805664
7 Precision std= 0.03677171817651272
8 Precision mean= 0.4164394749442808
8 Precision std= 0.10324497402064176
9 Precision mean= 0.8020319943349445
9 Precision std= 0.1013041515008211
10 Precision mean= 0.7064000182877034
10 Precision std= 0.06939506059932997
11 Precision mean= 0.8770294682065561
11 Precision std= 0.015897778270731723
12 Precision mean= 0.0
12 Precision std= 0.0
Recall mean of  0  = 0.9866006485198149
Recall std of  0  = 